In [7]:
import geemap
import ee

In [8]:
Map = geemap.Map()

In [9]:
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Image Collection

In [10]:
var LUUBWE_boundary = Wards.filter(ee.Filter.eq('WardNm2020','LUUBWE'));
var roi = ee.FeatureCollection(LUUBWE_boundary).geometry();
var LUUBWE_sentinel = Sentinel_2.filter(ee.Filter.bounds(roi))
.filter(ee.Filter.date('2019-01-01', '2019-12-01'))
.filter (ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5));

SyntaxError: invalid syntax (1575402935.py, line 1)

Cliping the Image and adding Map to Google Earth Enginee

In [ ]:
var rgbVis = {bands:['B3','B2','B1'], min: 0, max: 3000, gamma: 1.1};
var LUUBWE_sentinel_median = LUUBWE_sentinel.median();
var LUUBWE_sentinel_clip = LUUBWE_sentinel_median.clip(roi);
Map.addLayer(LUUBWE_boundary, {color:'red'});
Map.addLayer(LUUBWE_sentinel_clip, rgbVis);
Map.centerObject(LUUBWE_sentinel_clip, 10);

Area of the shapefile

In [ ]:
var  LUUBWE_Area =  LUUBWE_boundary.geometry().area();
var  LUUBWE_Area_sqKm = ee.Number(LUUBWE_Area).divide(1e6);
print( LUUBWE_Area_sqKm);

Supervised classification

Merge all the training points

In [ ]:
var lable = "Class";
var sample = Waterbody.merge(Builtup).merge(Cropland).merge(Vegetation).merge(Bareland);
var bands = ['B2','B3','B4','B5','B6','B7'];
var training = LUUBWE_sentinel_clip.select(bands).sampleRegions({
  collection: sample, 
  properties: ['Class'], 
  scale: 30
});
print('training', training);

Training dataset divided in two ways
Traing the model: 80%
Test the model: 20% data for testing

In [ ]:
var dataset = training.randomColumn();
var trainset = dataset.filter(ee.Filter.lessThan('random', 0.8));
var testset = dataset.filter(ee.Filter.greaterThanOrEquals('random', 0.8));


Create the Model CART

In [ ]:
var classifier = ee.Classifier.smileCart().train(trainset, lable, bands);

Classify the sentinel image

In [ ]:
var classified = LUUBWE_sentinel_clip.classify(classifier);
print('classified', classified);

Visualisation of the classification

In [ ]:
var landcoverpalette = [
  'blue',
  'red',
  'yellow',
  'green',
  'grey',
  ];
  
Map.addLayer(classified.clip(roi),{'min': 0, 'max': 4, 'palette': landcoverpalette});

Get information about the trained classifier.

In [ ]:
print('Results of trained classifier', classifier.explain());

Acuracy assement

Get a confusion matrix and overall accuracy for the training sample.

In [ ]:
var trainAccuracy = classifier.confusionMatrix();
print('Training error matrix', trainAccuracy);
print('Training overall accuracy', trainAccuracy.accuracy());

Get a confusion matrix and overall accuracy for the validation sample.

In [ ]:
testset = trainset.classify(classifier);
var validationAccuracy = testset.errorMatrix(lable, 'classification');
print('Validation error matrix', validationAccuracy);
print('Validation accuracy', validationAccuracy.accuracy());

Calculating Area by Class by Reagion
Calculating the Nested Group area of classes

In [ ]:
var areaImage = ee.Image.pixelArea().addBands(classified);
var areas = areaImage.reduceRegion({
  reducer: ee.Reducer.sum().group({
    groupField:1,
    groupName: 'class'}), 
    geometry: roi,
    scale: 10,
    maxPixels: 1e13});

print(areaImage);

Flatening the Nested Gropus areas of classes

In [ ]:
var classAreas = ee.List(areas.get('groups'));
var classAreaLists = classAreas.map(function(item){
  var areaDic = ee.Dictionary(item)
  var classNumber = ee.Number(areaDic.get('class')).format()
  var area = ee.Number(areaDic.get('sum')).divide(1e6).round()
  return ee.List([classNumber, area])
});

var results = ee.Dictionary(classAreaLists.flatten());
print(results);

Export Image

In [ ]:
Export.image.toDrive({
image:classified,
description: 'LUUBWE_sentinel_classified_2019',
fileFormat: 'GeoTIFF',
scale: 10,
region: roi,
});

Export table to your google drive

In [ ]:
Export.table.toDrive({
        collection: results,
        description: 'classAreaLists', 
        folder: 'EE',
    fileNamePrefix: 'classAreas', 
    fileFormat: 'CSV'
    });